### 1

#### 1.1

Securities Traded:
- Fixed-Income Instruments: Government bonds (U.S. Treasuries, JGBs), mortgage-backed securities.
- Derivatives: Interest-rate swaps, options on stock indices, futures contracts.
- Equities: Relative value trades like Royal Dutch/Shell share mispricings.
- Risk Arbitrage Positions: In mergers and acquisitions, targeting low-risk deals.


Trading Frequency:
- Long-Term Positions: Holding periods ranging from 6 months to over 2 years.
- Strategic Adjustments: Positions scaled up or down based on market conditions, not high-frequency trading.


Skewness:
- Many Small Wins: Aiming for consistent, incremental profits from minor market inefficiencies.
- Controlled Risk: Avoiding reliance on a few big hits; focused on steady accumulation.

    
Forecasting:

- Exploiting Mispricings: Utilizing sophisticated models to identify pricing discrepancies.
- Liquidity Provision: Anticipating and fulfilling institutional demand imbalances.
- Market Insight: Leveraging deep understanding of financial instruments and macroeconomic factors.

#### 1.2
- Elite Expertise: Led by industry pioneers with unmatched experience in arbitrage and trading strategies.
- Advanced Technology: Cutting-edge proprietary models and state-of-the-art software systems.
- Superior Financing: Access to favorable financing terms and low-cost leverage due to strong relationships with major financial institutions.
- Robust Risk Management: Innovative techniques including stress testing, diversified portfolios, and dynamic hedging.
- Significant Capital Base: Large equity capital enabling substantial positions and resilience against market fluctuations.
- Strategic Partnerships: Collaborations with global institutions providing valuable market insights and information flow.
- Operational Efficiency: Lean organizational structure with efficient back-office operations supporting complex trades.

#### 1.3

Collateral Haircuts:

- Proactive Liquidity Planning: Estimating worst-case haircuts and maintaining excess liquidity.
- Term Financing: Engaging in longer-term repos to mitigate sudden increases in haircuts.


Repo Maturity:
- Staggered Maturities: Diversifying repo expiration dates to avoid concentration risk.
- Extended Terms: Preferring repos with 6-12 months maturity to reduce rollover pressure.

    
Equity Redemption:
- Lock-Up Structures: Implementing initial three-year lock-ups, later modified to stagger withdrawals.
- Alignment of Interests: Principals reinvesting profits, showcasing commitment and stability.


Loan Access:
- Unsecured Term Loans: Securing long-term loans with no restrictive covenants.
- Reliable Credit Lines: Establishing a $700 million line of credit without material adverse change clauses.


#### 1.4

Enhanced Risk Models:
- Short-Term Volatility Adjustments: Incorporating liquidity-induced price movements into risk assessments.

Stress Testing Scenarios:
- Simulating Market Disruptions: Evaluating portfolio performance under extreme liquidity shortages.

Correlation Assumptions:
- Positive Short-Term Correlations: Assuming higher correlations during market stress to account for collective liquidity needs.
Liquidity Buffer Maintenance:

-Excess Capital Reserves: Ensuring sufficient funds to meet collateral calls without forced liquidation.

Term Financing Strategies:
- Reducing Rollover Risk: Using long-term repos to lessen dependence on short-term funding markets.


#### 1.5
Focused on Risk, Not Leverage Ratios:
- Holistic Risk Management: Prioritizing the actual economic risk over simplistic leverage metrics.

Comparable Leverage Levels:
- Industry Standards: Operating at leverage ratios similar to major banks and financial firms.

  
Mitigated Through Hedging:
- Offsetting Positions: Employing long-short strategies to neutralize market exposures.


Transparent Disclosures:
- Regulatory Compliance: Providing necessary information to lenders and regulators to maintain trust.


#### 1.6

- Liquidity Crunch: - Margin Calls and Collateral Demands
- Extended Divergence - Persistent Mispricing:
- Market Disruptions: Systemic Events
- Leverage Amplification: Magnified Losses
- Funding Constraints - Capital Depletion
- Model Limitations - Overreliance on Historical Data
- Forced Liquidations - Fire Sales

### 2

In [119]:
import pandas as pd
import numpy as np
from tabulate import tabulate
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

In [121]:
df_rfr = pd.read_excel('../data/gmo_data.xlsx', sheet_name='risk-free rate').set_index('date')/12
df_ltcm_returns = pd.read_excel('../data/ltcm_exhibits_data.xlsx', sheet_name='Exhibit 2', skiprows=2, index_col=0, parse_dates=[0]).iloc[:-4, :]
df_ltcm_returns.index.name = 'Date'
df_ltcm_returns.index = pd.to_datetime(df_ltcm_returns.index) + pd.tseries.offsets.MonthEnd(0)
df_ltcm_returns = df_ltcm_returns[['Net Monthly Performanceb', 'Gross Monthly Performancea']].dropna().rename(columns={'Net Monthly Performanceb': 'LTCM Net', 'Gross Monthly Performancea': 'LTCM Gross'})
df_spy = pd.read_excel('../data/gmo_data.xlsx', sheet_name='total returns').set_index('date')[['SPY']]
df = df_ltcm_returns.join(df_rfr, how='inner').join(df_spy, how='inner')
df.loc[:, ['LTCM Net', 'LTCM Gross', 'SPY']] = df.loc[:, ['LTCM Net', 'LTCM Gross', 'SPY']].subtract(df['TBill 3M'], axis=0)

In [122]:
df.head()

,LTCM Net,LTCM Gross,TBill 3M,SPY
1996-12-31,0.024691,0.030691,0.004309,-0.027601
1997-01-31,0.037711,0.056711,0.004289,0.057497
1997-02-28,0.000650,0.001650,0.004350,0.005215
1997-03-31,0.011565,0.016565,0.004435,-0.050156
1997-04-30,0.005639,0.013639,0.004361,0.060008


In [123]:
# 2.1
def calculate_univariate_performance_statistics(df, annualization_factor=1, quantile=0.05, periods=None):
    if periods is None:
        periods = [(df.index.min(), df.index.max())]
    
    summary_list = []

    for period in periods:
        start_date, end_date = period
        period_df = df.loc[start_date:end_date]
        
        summary_df = pd.DataFrame(index=period_df.columns)
        summary_df['Mean'] = period_df.mean() * annualization_factor
        summary_df['Volatility'] = period_df.std() * (annualization_factor ** 0.5)
        summary_df['Sharpe_ratio'] = summary_df['Mean'] / summary_df['Volatility']
        summary_df['Skweness'] = period_df.skew() 
        summary_df['Volatility'] = period_df.kurtosis()
        summary_df[f'VaR({quantile})'] = period_df.quantile(quantile, axis=0)
        
        start_year = pd.to_datetime(start_date).year
        end_year = pd.to_datetime(end_date).year
        summary_df['Period'] = f"{start_year}-{end_year}"
        
        summary_df = summary_df.reset_index().rename(columns={'index': 'Factor'}).set_index(['Period', 'Factor'])
        summary_list.append(summary_df)
    
    summary = pd.concat(summary_list)
    return summary

summary_statistics = calculate_univariate_performance_statistics(df, 12)
summary_statistics.T

Period       1996-1998                                 
Factor        LTCM Net LTCM Gross    TBill 3M       SPY
Mean          0.012582   0.080296    0.051704  0.222922
Volatility    7.800657   5.952458    0.131893 -1.014941
Sharpe_ratio  0.106070   0.610928  179.142799  1.676513
Skweness     -2.479688  -2.033004   -0.017451 -0.357027
VaR(0.05)    -0.048730  -0.051330    0.004199 -0.036298

- LTCM Gross shows impressive returns with efficient risk management pre-fees, evidenced by a strong Sharpe Ratio.
- LTCM Net reflects the substantial impact of fees, reducing both returns and risk-adjusted performance.
- High Negative Skewness across LTCM strategies signals a vulnerability to extreme market downturns.
- Comparative Metrics highlight LTCM’s unique position between low-risk TBill investments and high-return SPY equities, offering a blend of both but with added complexities and risks.

In [129]:
# 2.2.a
def calculate_univariate_regression(df_endo, df_exo, annualization=1):
    alphas, r_squared = [], []
    betas_dict = {factor: [] for factor in df_exo.columns} 

    for col in df_endo.columns:
        y = df_endo[col]
        X = sm.add_constant(df_exo)
        model = sm.OLS(y, X, missing='drop').fit()
        
        alpha = model.params['const'] * annualization
        
        for factor in df_exo.columns:
            betas_dict[factor].append(model.params.get(factor, np.nan))
        
        alphas.append(alpha)
        r_squared.append(model.rsquared)

    results_df = pd.DataFrame({
        'Alpha': alphas,
        'R-Squared': r_squared,
    }, index=df_endo.columns)

    for factor, beta_values in betas_dict.items():
        results_df[f'Beta_{factor}'] = beta_values
    
    return results_df

In [131]:
# 2.2.a
calculate_univariate_regression(df[['LTCM Net']], df[['SPY']]).T

,LTCM Net
Alpha,0.002755
R-Squared,0.010602
Beta_SPY,-0.091855


#### 2.2.b
No, LTCM does not appear to be a closet indexer.

- A low R² indicates ndicates minimal correlation with the SPY index.
- Negative Beta suggests inverse relationship with the benchmark, contrary to index mirroring.
- The negligible R² and negative beta demonstrate that LTCM's performance is largely independent and even counter to the SPY index, indicating genuine active management rather than passive replication.

#### 2.2.c
Yes, LTCM appears to deliver excess returns beyond the expected risk premium from market exposure.

- Positive Alpha suggests LTCM generates returns above what would be expected from its market exposure alone.
- Sharpe Ratio indicates decent risk-adjusted returns, especially pre-fees.
-  Most of LTCM's returns are not explained by market movements, implying skill or unique strategies.
Conclusion:

The positive alpha, despite low correlation with the SPY index, indicates that LTCM's strategies are adding value beyond mere market risk factors.


In [141]:
# 2.3.a
df_spy_2 = (df[['SPY']]**2).rename(columns={'SPY':'SPY^2'})
calculate_univariate_regression(df[['LTCM Net']], df[['SPY']].join(df_spy_2)).T

                                

,LTCM Net
Alpha,-0.003779
R-Squared,0.049283
Beta_SPY,-0.225593
Beta_SPY^2,5.263789


#### 2.3.b
Increase in R²:
- Linear Model R²: 1.06%
- Quadratic Model R²: 4.93%

The inclusion of the quadratic term substantially increases the explanatory power. The model now captures additional non-linear relationships between LTCM’s returns and market movements.

#### 2.3.c

- Negative β₁ exposure suggests an inverse relationship with market returns.
- Positive β₂ exposure indicates sensitivity to the squared market returns.

- Positive β₂: Aligns with long option positions, as positive convexity indicates profiting from increased volatility.
- Negative β₁: Suggests LTCM gains when the market falls, similar to holding put options.

#### 2.3.d
LTCM is Positively Exposed to Market Volatility.

- Positive Quadratic Coefficient indicates that LTCM’s returns increase with the square of market returns.
- Positive Exposure to Volatility

The significant positive β₂ captures the enhanced returns from increased volatility, characteristic of long option strategies.

In [144]:
# 2.4.a
k1 = 0.03
k2 = -0.03
df['u'] = np.maximum(df['SPY'] - k1, 0)
df['d'] = np.maximum(k2 - df['SPY'], 0)

calculate_univariate_regression(df[['LTCM Net']], df[['SPY', 'u', 'd']].join(df_spy_2)).T


,LTCM Net
Alpha,0.025579
R-Squared,0.306496
Beta_SPY,-1.065809
Beta_u,10.166784
Beta_d,5.268855
Beta_SPY^2,-69.036581


#### 2.4.b
Call-like Factor (u):
Position: Long
- Positive βᵤ indicates LTCM benefits from upward market movements.
Put-like Factor (d):
Position: Long
-  Positive β_d signifies LTCM gains from downward market movements.

#### 2.4.c

Call-like factor drives LTCM’s returns more significantly than the put-like factor.
- βᵤ (10.1668) vs. β_d (5.2689)
- Greater sensitivity to upward market movements compared to downward movements.


#### 2.4.d

LTCM is positively exposed to market volatility, primarily through long positions on the market’s upside while also benefiting from downside movements.

- Positive Exposure to Volatility: Long on Call-like (u) and Put-like (d) Factors
- Emphasis on Upside: Greater Exposure via Call-like Factor: 
- Strategic Positioning: Option-like Behavior

In [151]:
df_rfrs = pd.read_excel('../data/fx_rf_data.xlsx', sheet_name='risk-free rates').set_index('date')
df_rfrs_levels = pd.DataFrame(index=df_rfrs.index)
for col in df_rfrs.columns:
    df_rfrs_levels['log_' + col] = np.log(1+df_rfrs[col])

df_exchange_rates = pd.read_excel('../data/fx_rf_data.xlsx', sheet_name='exchange rates').set_index('date')
df_exchange_levels = pd.DataFrame(index=df_exchange_rates.index)
for col in df_exchange_rates.columns:
    df_exchange_levels['log_' + col] = np.log(1+df_exchange_rates[col])

In [152]:
df_rfrs_levels.head()

,log_USD,log_JPY,log_EUR,log_GBP,log_MXN,log_CHF
date,,,,,,
2008-01-01,0.043059,0.004769,0.041622,0.055435,0.072321,0.020435
2008-01-02,0.040722,0.004769,0.035367,0.053304,0.072321,0.020435
2008-01-03,0.042545,0.004769,0.038644,0.054488,0.072321,0.021272
2008-01-04,0.041322,0.004769,0.039846,0.053541,0.072321,0.020734
2008-01-07,0.041322,0.004918,0.040182,0.053162,0.072321,0.019999


In [153]:
df_exchange_levels.head()

,log_JPY,log_EUR,log_GBP,log_MXN,log_CHF
date,,,,,
2008-01-01,0.008917,0.899836,1.094069,0.087855,0.632548
2008-01-02,0.009078,0.904825,1.092192,0.087663,0.638321
2008-01-03,0.009106,0.906240,1.088899,0.087892,0.641959
2008-01-04,0.009167,0.905958,1.089908,0.087590,0.643011
2008-01-07,0.009117,0.904056,1.088697,0.087791,0.639588


In [182]:
holdings = []
for col in df_exchange_levels.columns:
    df_holding_excess = (df_exchange_levels[col] - df_exchange_levels[col].shift(1) + df_rfrs_levels[col].shift(1) - df_rfrs_levels['log_USD'].shift(1)).to_frame().rename(columns={0:col[4:]})
    summary_statistics = calculate_univariate_performance_statistics(df_holding_excess)
    holdings.append(summary_statistics)
df_holdings = holdings[0].T
for i in range(1, len(holdings)):
    df_holdings = df_holdings.join(holdings[i].T)
df_holdings

Period       2008-2024                                        
Factor             JPY       EUR       GBP       MXN       CHF
Mean         -0.011641 -0.006755 -0.000135  0.046101 -0.012875
Volatility    1.095299  0.152606  2.983454 -0.501791  0.336896
Sharpe_ratio -0.732502 -0.581863 -0.014855  4.394064 -0.960570
Skweness     -1.517629  0.171816  0.768878  0.008631 -0.542781
VaR(0.05)    -0.051385 -0.026630 -0.015338  0.027877 -0.037818

- Only MXN stands out with positive returns and high Sharpe, making it the prime candidate for carry trades.
- GBP and JPY exhibit high volatility but negative returns, making them unattractive despite potential for high rewards.
- EUR and CHF offer manageable volatility but fail to deliver positive excess returns, limiting their appeal.


#### 3.2.1

None of the currencies exhibit a mean excess return of zero, directly contradicting UIP which posits that expected excess returns should balance out to zero due to interest rate differentials.

#### 3.2.2
MXN (Mexican Peso)
- Sharpe Ratio: 4.39
- MXN offers the highest Sharpe ratio, indicating superior risk-adjusted returns compared to other currencies, making it the optimal choice for a long carry trade position.

#### 3.2.3

Underperforming Currencies:
- JPY 
- EUR 
- GBP 
- CHF

Positive Performer:
- MXN

All currencies except MXN earned negative or near-zero excess returns in USD terms, with JPY and CHF showing significant negative returns, highlighting their unfavorable carry trade conditions.

In [191]:
from tabulate import tabulate

results = []
for col in df_exchange_levels.columns:
    df_endo = (df_exchange_levels[col].diff()).to_frame().rename(columns={0:col[4:]})
    df_exo = (df_rfrs_levels['log_USD'] - df_rfrs_levels[col]).shift(1).to_frame().rename(columns={0:col[4:]})
    results.append(calculate_univariate_regression(df_endo, df_exo))
for result in results:
    print(tabulate(result.T, headers=result.T.columns, tablefmt="heavy_grid"))


┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃           ┃      log_JPY ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ Alpha     ┃ -7.57386e-07 ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ R-Squared ┃  2.52936e-05 ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ Beta_JPY  ┃  1.8891e-05  ┃
┗━━━━━━━━━━━┻━━━━━━━━━━━━━━┛
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃           ┃      log_EUR ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ Alpha     ┃ -7.03973e-05 ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ R-Squared ┃  0.000311803 ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ Beta_EUR  ┃  0.00493977  ┃
┗━━━━━━━━━━━┻━━━━━━━━━━━━━━┛
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃           ┃      log_GBP ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ Alpha     ┃ -6.17648e-05 ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ R-Squared ┃  0.00175938  ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ Beta_GBP  ┃  0.0173741   ┃
┗━━━━━━━━━━━┻━━━━━━━━━━━━━━┛
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━┓
┃           ┃      log_MXN ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ Alpha     ┃ -1.62713e-05 ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ R-Squared ┃  1.16728e-05 ┃
┣━━━━━━━━━━━╋━━━━━━━━━━━━━━┫
┃ Beta_MXN  ┃ 

#### 3.3.2.a
Predicting Relative Strengthening of the USD

Currencies:
JPY (Japanese Yen)
EUR (Euro)
GBP (British Pound)
CHF (Swiss Franc)
- All listed currencies have positive β coefficients
- As foreign risk-free rates rise, these currencies are expected to depreciate relative to USD, leading to a strengthening of the USD against them.

#### 3.3.2.b
Predicting Relative Weakening of the USD

Currency:
- MXN (Mexican Peso)

 MXN has a negative β coefficient. An increase in the foreign risk-free rate for MXN suggests that the MXN may appreciate relative to USD, resulting in a weakening of the USD against MXN.

#### 3.3.2.c

Strongest FX Predictability

Currency:
- GBP (British Pound)

GBP exhibits the highest β coefficient , indicating the most sensitive response to changes in foreign risk-free rates. Although still low R2, GBP's higher β relative to other currencies suggests greater predictability in USD's movement against GBP when foreign rates shift.


In [195]:
alpha_list = []
beta_list = []
currency_list = []

for df in results:
    currency = df.index[0].split('_')[-1]
    
    # Extract Alpha
    alpha = df.loc[df.index[0], 'Alpha']
    
    # Extract Beta (dynamic column name based on currency)
    beta_col = f'Beta_{currency}'
    beta = df.loc[df.index[0], beta_col]
    
    # Append to lists
    currency_list.append(currency)
    alpha_list.append(alpha)
    beta_list.append(beta)

# Create summary DataFrame
fx_hldg_reg_summary = pd.DataFrame({
    'Alpha': alpha_list,
    'Beta': beta_list
}, index=currency_list)

# Display the summary
print(fx_hldg_reg_summary)


            Alpha      Beta
JPY -7.573863e-07  0.000019
EUR -7.039732e-05  0.004940
GBP -6.176476e-05  0.017374
MXN -1.627131e-05 -0.000160
CHF  7.938598e-06  0.001790


In [197]:
# Initialize list to store premium data
fx_prem_lst = []

# Iterate through each currency
for currency in fx_hldg_reg_summary.index:
    # Calculate interest rate differential: (rf_USD - rf_foreign), shifted by 1 period
    interest_rate_diff = (df_rfrs_levels['log_USD'].shift(1) - df_rfrs_levels[f'log_{currency}'].shift(1))
    
    # Retrieve Alpha and Beta
    alpha = fx_hldg_reg_summary.loc[currency, 'Alpha']
    beta = fx_hldg_reg_summary.loc[currency, 'Beta']
    
    # Compute Expected FX Risk Premium
    expected_fx_premium = alpha + (beta - 1) * interest_rate_diff
    
    # Drop NaN values resulting from the shift
    expected_fx_premium = expected_fx_premium.dropna()
    
    # Calculate the number of positive premiums
    positive_premium = (expected_fx_premium > 0).sum()
    total_months = expected_fx_premium.shape[0]
    frequency_positive = (positive_premium / total_months) * 100  # Percentage
    
    # Append results to the list
    fx_prem_lst.append({
        'Currency': currency,
        'Months - Positive Premium': positive_premium,
        'Total Months': total_months,
        'Frequency(%)-Positive Premium': frequency_positive
    })

# Create DataFrame from the list
fx_premium = pd.DataFrame(fx_prem_lst).set_index('Currency')

# Display the premium frequencies
fx_premium


,Months - Positive Premium,Total Months,Frequency(%)-Positive Premium
Currency,,,
JPY,8,4392,0.182149
EUR,1149,4392,26.161202
GBP,2316,4392,52.732240
MXN,4392,4392,100.000000
CHF,115,4392,2.618397
